In [1]:
import os
import codecs
import spacy

In [2]:
os.chdir(r'E:\MachineLearning\NLP\yelp_dataset\dataset')

In [3]:
businesses_filepath = os.getcwd() + r'\business.json'

In [4]:
review_json_filepath = os.getcwd() + r'\review.json'

In [5]:
with codecs.open(businesses_filepath, encoding='utf_8') as f:
    first_business_record = f.readline() 

In [6]:
print (first_business_record)

{"business_id": "FYWN1wneV18bWNgQjJ2GNg", "name": "Dental by Design", "neighborhood": "", "address": "4855 E Warner Rd, Ste B9", "city": "Ahwatukee", "state": "AZ", "postal_code": "85044", "latitude": 33.3306902, "longitude": -111.9785992, "stars": 4.0, "review_count": 22, "is_open": 1, "attributes": {"AcceptsInsurance": true, "ByAppointmentOnly": true, "BusinessAcceptsCreditCards": true}, "categories": ["Dentists", "General Dentistry", "Health & Medical", "Oral Surgeons", "Cosmetic Dentists", "Orthodontists"], "hours": {"Friday": "7:30-17:00", "Tuesday": "7:30-17:00", "Thursday": "7:30-17:00", "Wednesday": "7:30-17:00", "Monday": "7:30-17:00"}}



In [7]:
with codecs.open(review_json_filepath, encoding='utf_8') as f:
    first_review_record = f.readline() 

In [8]:
print (first_review_record)

{"review_id":"v0i_UHJMo_hPBq9bxWvW4w","user_id":"bv2nCi5Qv5vroFiqKGopiw","business_id":"0W4lkclzZThpx3V65bVgig","stars":5,"date":"2016-05-28","text":"Love the staff, love the meat, love the place. Prepare for a long line around lunch or dinner hours. \n\nThey ask you how you want you meat, lean or something maybe, I can't remember. Just say you don't want it too fatty. \n\nGet a half sour pickle and a hot pepper. Hand cut french fries too.","useful":0,"funny":0,"cool":0}



In [9]:
import json

restaurant_ids = set()

# open the businesses file
with codecs.open(businesses_filepath, encoding='utf_8') as f:
    
    # iterate through each line (json record) in the file
    for business_json in f:
        
        # convert the json record to a Python dict
        business = json.loads(business_json)
        
        # if this business is not a restaurant, skip to the next one
#        if u'Dentists' not in business[u'categories']:
        if u'Restaurants' not in business[u'categories']:
            continue
            
        # add the restaurant business id to our restaurant_ids set
        restaurant_ids.add(business[u'business_id'])

# turn restaurant_ids into a frozenset, as we don't need to change it anymore
restaurant_ids = frozenset(restaurant_ids)

# print the number of unique restaurant ids in the dataset
print ('{:,}'.format(len(restaurant_ids)), 'restaurants in the dataset.')

54,618 restaurants in the dataset.


In [10]:
review_json_filepath = os.getcwd() + r'\review.json'
review_txt_filepath = os.getcwd() + r'\review.txt'

In [11]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 0 == 1:
    
    review_count = 0

    # create & open a new file in write mode
    with codecs.open(review_txt_filepath, 'w', encoding='utf_8') as review_txt_file:

        # open the existing review json file
        with codecs.open(review_json_filepath, encoding='utf_8') as review_json_file:

            # loop through all reviews in the existing file and convert to dict
            for review_json in review_json_file:
                review = json.loads(review_json)

                # if this review is not about a restaurant, skip to the next one
                if review[u'business_id'] not in restaurant_ids:
                    continue

                # write the restaurant review as a line in the new file
                # escape newline characters in the original review text
                review_txt_file.write(review[u'text'].replace('\n', '\\n') + '\n')
                review_count += 1

    print (u'''Text from {:,} restaurant reviews
              written to the new txt file.'''.format(review_count))
    
else:
    
    with codecs.open(review_txt_filepath, encoding='utf_8') as review_txt_file:
        for review_count, line in enumerate(review_txt_file):
            pass
    print (u'Text from {:,} restaurant reviews in the txt file.'.format(review_count + 1))

review_txt_file.close()
#review_json_file.close()

Text from 3,223,214 restaurant reviews in the txt file.
Wall time: 56.3 s


In [12]:
import spacy
from spacy import load
import pandas as pd
import itertools as it

nlp = spacy.load('en')

In [13]:
with codecs.open(review_txt_filepath, encoding='utf_8') as f:
    sample_review = ''.join(list(it.islice(f, 5, 9)))
    sample_review = sample_review.replace('\\n', '\n')
        
print(sample_review)

Had their chocolate almond croissant and it was amazing! So light and buttery and oh my how chocolaty.

If you're looking for a light breakfast then head out here. Perfect spot for a coffee/latté before heading out to the old port
Who would have guess that you would be able to get fairly decent Vietnamese restaurant in East York? 

Not quite the same as Chinatown in terms of pricing (slightly higher) but definitely one of the better Vietnamese restaurants outside of the neighbourhood. When I don't have time to go to Chinatown, this is the next best thing as it is down the street from me.

So far the only items I have tried are the phos (beef, chicken & vegetarian) - and they have not disappointed me! Especially the chicken pho.

Next time I go back, I'm going to try the banh cuon (steamed rice noodle) and the vermicelli!
Not bad!! Love that there is a gluten-free, vegan version of the cheese curds and gravy!!

Haven't done the poutine taste test yet with smoke's but Im excited to see w

In [14]:
%%time
parsed_review = nlp(sample_review)

Wall time: 1.9 s


In [15]:
print(parsed_review)

Had their chocolate almond croissant and it was amazing! So light and buttery and oh my how chocolaty.

If you're looking for a light breakfast then head out here. Perfect spot for a coffee/latté before heading out to the old port
Who would have guess that you would be able to get fairly decent Vietnamese restaurant in East York? 

Not quite the same as Chinatown in terms of pricing (slightly higher) but definitely one of the better Vietnamese restaurants outside of the neighbourhood. When I don't have time to go to Chinatown, this is the next best thing as it is down the street from me.

So far the only items I have tried are the phos (beef, chicken & vegetarian) - and they have not disappointed me! Especially the chicken pho.

Next time I go back, I'm going to try the banh cuon (steamed rice noodle) and the vermicelli!
Not bad!! Love that there is a gluten-free, vegan version of the cheese curds and gravy!!

Haven't done the poutine taste test yet with smoke's but Im excited to see w

In [16]:
for num, sentence in enumerate(parsed_review.sents):
    print ('Sentence {}:'.format(num + 1))
    print (sentence)
    print ('')

Sentence 1:
Had their chocolate almond croissant and it was amazing!

Sentence 2:
So light and buttery

Sentence 3:
and oh my how chocolaty.



Sentence 4:
If you're looking for a light breakfast then head out here.

Sentence 5:
Perfect spot for a coffee/latté before heading out to the old port
Who would have guess that you would be able to get fairly decent Vietnamese restaurant in East York? 



Sentence 6:
Not quite the same as Chinatown in terms of pricing (slightly higher) but definitely one of the better Vietnamese restaurants outside of the neighbourhood.

Sentence 7:
When I don't have time to go to Chinatown, this is the next best thing as it is down the street from me.



Sentence 8:
So far the only items I have tried are the phos (beef, chicken & vegetarian) - and they have not disappointed me!

Sentence 9:
Especially the chicken pho.



Sentence 10:
Next time I go back, I'm going to try the banh cuon (steamed rice noodle) and the vermicelli!


Sentence 11:
Not bad!!

Sentenc

In [17]:
for num, entity in enumerate(parsed_review.ents):
    print ('Entity {}:'.format(num + 1), entity, '-', entity.label_)
    print ('')

Entity 1: 
 - GPE

Entity 2: Vietnamese - NORP

Entity 3: East York - GPE

Entity 4: one - CARDINAL

Entity 5: Vietnamese - NORP

Entity 6: 
 - GPE

Entity 7: 
 - GPE

Entity 8: 6 - CARDINAL

Entity 9: 9 - CARDINAL

Entity 10: 
 - GPE



In [18]:
import pandas as pd
token_text = [token.orth_ for token in parsed_review]
token_pos = [token.pos_ for token in parsed_review]

pd.DataFrame(list(zip(token_text, token_pos)),columns=['token_text', 'part_of_speech'])

,token_text,part_of_speech
0,Had,VERB
1,their,ADJ
2,chocolate,NOUN
3,almond,NOUN
4,croissant,ADJ
5,and,CCONJ
6,it,PRON
7,was,VERB
8,amazing,ADJ
9,!,PUNCT


In [19]:
token_lemma = [token.lemma_ for token in parsed_review]
token_shape = [token.shape_ for token in parsed_review]

pd.DataFrame(list(zip(token_text, token_lemma, token_shape)),
             columns=['token_text', 'token_lemma', 'token_shape'])

,token_text,token_lemma,token_shape
0,Had,have,Xxx
1,their,-PRON-,xxxx
2,chocolate,chocolate,xxxx
3,almond,almond,xxxx
4,croissant,croissant,xxxx
5,and,and,xxx
6,it,-PRON-,xx
7,was,be,xxx
8,amazing,amazing,xxxx
9,!,!,!


In [20]:
token_entity_type = [token.ent_type_ for token in parsed_review]
token_entity_iob = [token.ent_iob_ for token in parsed_review]

pd.DataFrame(list(zip(token_text, token_entity_type, token_entity_iob)),
             columns=['token_text', 'entity_type', 'inside_outside_begin'])

,token_text,entity_type,inside_outside_begin
0,Had,,O
1,their,,O
2,chocolate,,O
3,almond,,O
4,croissant,,O
5,and,,O
6,it,,O
7,was,,O
8,amazing,,O
9,!,,O


In [21]:
token_attributes = [(token.orth_,
                     token.prob,
                     token.is_stop,
                     token.is_punct,
                     token.is_space,
                     token.like_num,
                     token.is_oov)
                    for token in parsed_review]

In [22]:
df = pd.DataFrame(token_attributes,
                  columns=['text',
                           'log_probability',
                           'stop?',
                           'punctuation?',
                           'whitespace?',
                           'number?',
                           'out of vocab.?'])

In [23]:
df.loc[:, 'stop?':'out of vocab.?'] = (df.loc[:, 'stop?':'out of vocab.?']
                                       .applymap(lambda x: u'Yes' if x else u''))

In [24]:
df

,text,log_probability,stop?,punctuation?,whitespace?,number?,out of vocab.?
0,Had,-20.0,,,,,Yes
1,their,-20.0,Yes,,,,Yes
2,chocolate,-20.0,,,,,Yes
3,almond,-20.0,,,,,Yes
4,croissant,-20.0,,,,,Yes
5,and,-20.0,Yes,,,,Yes
6,it,-20.0,Yes,,,,Yes
7,was,-20.0,Yes,,,,Yes
8,amazing,-20.0,,,,,Yes
9,!,-20.0,,Yes,,,Yes


In [25]:
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

C:\Users\partha\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [36]:
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    with codecs.open(filename, encoding='utf_8') as f:
        i = 0
        for review in f:
            i = i + 1
            if i < 200:
                yield review.replace('\\n', '\n')
            else:
                break
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_review in nlp.pipe(line_review(filename),
                                  batch_size=10000, n_threads=4):
        
        for sent in parsed_review.sents:
            yield' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

In [37]:
unigram_sentences_filepath = os.getcwd() + r'\unigram_sentences_all.txt'

In [38]:

%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 0 == 0:

    with codecs.open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
        for sentence in lemmatized_sentence_corpus(review_txt_filepath):
            f.write(sentence + '\n')



Wall time: 7.86 s


In [39]:


unigram_sentences = LineSentence(unigram_sentences_filepath)



In [42]:
for unigram_sentence in it.islice(unigram_sentences, 5, 9):
    print (u' '.join(unigram_sentence))
    print (u'')

hand cut french fry too

super simple place but amazing nonetheless

-PRON- be be around since the 30 's and -PRON- still serve the same thing -PRON- start with a bologna and salami sandwich with mustard

staff be very helpful and friendly

